In [1]:
from hello.hello3 import HelloApp, NotLoggedInError
import datetime
from hello import hello_logger
import os
import requests
import time

In [2]:
# General functions

def call(f, *a, _quiet=False, **k):
    if not _quiet:
        now = datetime.datetime.now().strftime("%H:%M:%S")
        s = "(" + ", ".join(map(repr, a)) + ")"
        print("%s Calling %s%s" % (now, f.__name__, s))
    while True:
        try:
            return f(*a, **k)
        except NotLoggedInError:
            h.login()
        except requests.exceptions.Timeout as e:
            print("Got timeout! Make sure you're on the right network!")
        except (BadError, IOError, requests.exceptions.ConnectionError) as e:
            print("Error occurred:", e.__class__)
        else:
            return rv        
            
def connect(ip, *args):
    global h
    h = HelloApp(ip, *args)
    
def dfmt(d):
    return d.strftime("%m/%d/%Y %I:%M:%S %p")
    
def wait1(s):
    if s <= 0: return
    dend = dfmt(datetime.datetime.now() + datetime.timedelta(seconds=s))
    print("Sleeping %d seconds, ending at %s" %(s,dend))
    start = time.time()
    end = start + s
    while time.time() < end:
        time.sleep(5)
    
def wait3(s, src):
    start = datetime.datetime.now()
    end = start + datetime.timedelta(seconds=s)
    m1 = dfmt(start)
    m2 = dfmt(end)
    print("\r(%s) Sleeping %d seconds from %s to %s          " % (src, s, m1, m2), end="")
    tend = time.time() + s
    while time.time() < tend:
        time.sleep(5)
    
def wait2(s):
    pass
wait = wait1

In [3]:
# Specific functions

# globals
do_o2_req  = 0
do_n2_req  = 0
ph_co2_req = 0
batch_tmpl = "domcr t %d"
logger = None

def setconfig(g, s, v):
    call(h.setconfig, g, s, v)

def getmv():
    return call(h.gpmv, _quiet=True)
    
def setup_gases():
    global do_o2_req, do_n2_req, ph_co2_req
    setconfig("pH", 'CO2_Manual_Max_(%)', 100)
    setconfig("DO", 'O2_Manual_Max_(%)', 100)
    setconfig("DO", 'N2_Manual_Max_(%)', 100)
    mv = getmv()
    do_o2_req = mv['do']['outputUp']
    do_n2_req = mv['do']['outputDown']
    ph_co2_req = mv['ph']['outputDown']
    
def setup_controls(rpm=18, temp=37):
    setag(rpm)
    settemp(temp)   
    
def maingas(v):
    qset(h.setmg, v)
    
def qset(f, v):
    if v > 0:
        call(f, 1, v)
    else:
        call(f, 2, 0)
        
def main_n2(pc):
    setdo(pc, do_o2_req)
    
def micro_n2(pc):
    global ph_co2_req
    if pc > 0:
        call(h.setph, 1, pc, 0)
    else:
        call(h.setph, 2, ph_co2_req, 0)
    ph_co2_req = pc
    
def phoff():
    call(h.setph, 2, 0, 0)
    
def micro_o2(pc):
    setdo(do_n2_req, pc)
    
def setdo(n2, o2):
    global do_n2_req, do_o2_req
    call(h.setdo, 1, n2, o2)
    do_o2_req = o2
    do_n2_req = n2
    
def setag(rpm):
    call(h.setag, 0, rpm)
    
def settemp(t):
    call(h.settemp, 0, t)
    
def endbatch():
    if call(h.batchrunning):
        call(h.endbatch)
    
def startbatch(n):
    endbatch()
    call(h.startbatch, n)
    
def docal1pt():
    v = call(h.getrawvalue, "doa")
    call(h.trycal, 'doa', v, 100)
    call(h.savetrialcal, 'doa')

def dopv():
    return getmv()['do']['pv']

def waitdo_lower(t, timeout=8*3600):
    while dopv() > t and timeout>0:
        wait3(60, 'waitdo_lower')
        timeout -= 60
    print()
    
def dooff():
    call(h.setdo, 2, 0, 0)
    
def waitdo_higher(t, timeout=8*3600):
    while dopv() < t and timeout>0:
        wait3(60, 'waitdo_higher')
        timeout -= 60
    print()
    
def rampdown(mg=0.5, min=20, timeout=2*3600):
    print("rampdown(%r, %r, %r)"%(mg, min, timeout))
    maingas(mg)
    micro_n2(100)
    dooff()
    waitdo_lower(min, timeout)
    phoff()
    
def flushheadspace(gas='air', mg=0.5, wt=5*60):
    print("flushheadspace(%r, %r, %r)"%(gas, mg, wt))
    maingas(mg)
    if gas == "n2":
        main_n2(100)
    else:
        if do_n2_req != 0 or do_o2_req != 0:
            dooff()
    wait(wt)
    dooff()
    
def rampup(mg=0.5, o2=20, max=80, timeout=2*3600):
    print("rampup(%r, %r, %r, %r)"%(mg, o2, max, timeout))
    maingas(mg)
    micro_o2(o2)
    micro_n2(0)
    main_n2(0)
    waitdo_higher(max, timeout)
    dooff()
    
def newbatch(i, j):
    startbatch(batch_tmpl%(i,j))

In [4]:
def setup_logger(interval=10):
    global logger
    keys = [
        'temperature.pv',
        'agitation.pv',
        'ph.outputDown',
        'ph.manDown',
        'do.manUp',
        'do.outputDown',
        'do.pv',
        'do.manDown',
        'do.outputUp',
        'maingas.man',
        'maingas.mode',
        'maingas.pv',
        'MFCs.air',
        'MFCs.n2',
        'MFCs.o2',
        'MFCs.co2',
        'level.pv'
    ]
    if logger is not None:
        logger.stop()
    logger = hello_logger.HelloLogger(h, interval, None, keys)
    logger.start()
    
def logfile(fn):
    print("Creating new file: %r"%fn)
    logger.new_file(fn)
    
def endlog():
    logger.new_file(None)

In [5]:
def calibrate(t):
    maingas(0.5)
    micro_o2(100)
    micro_n2(0)
    main_n2(0)
    if t < 5400:
        wait(t/2)
        micro_o2(0)
        wait(t/2)
    else:
        wait(t-3600)
        micro_o2(0)
        wait(3600)
    docal1pt()
    
def ntests(n):
    return range(1,n+1)

### Test Utilities

In [6]:
fn_tmpl = "DO micro test %s %s %s.csv"

def new_file(typ, name):
    logfile(fn_tmpl% (typ, name, bt()))

def dotest(mg, o2, i):
    new_file("rampdown", i)
    rampdown(mg)
    endlog()
    flushheadspace('air')
    new_file("rampup", i)
    rampup(mg, o2)
    endlog()
    dooff()
    
def bt(): 
    return datetime.datetime.now().strftime("%y%m%d")

def mkfldr(p):
    path = "C:\\Users\\natha\\Documents\\test\\"+p
    os.makedirs(path, exist_ok=True)
    os.chdir(path)
    
def chd(p):
    os.chdir("C:\\Users\\natha\\Documents\\test\\"+p)

### Pre-Test Setup

In [ ]:
connect('192.168.1.81')
setup_gases()
setup_controls(32, 37)
setup_logger()
mkfldr("Sam Microsparger Test")

In [ ]:
calibrate(1*3600)
dotest(0.1, 100, "dip tube 2")    

In [7]:
connect('192.168.1.81')
setup_gases()
setup_controls(32, 37)
setup_logger()
chd("Sam Microsparger Test")
for i in range(1,4):
    dotest(0.1, 100, "L plate A %d"%i)    

15:53:46 Calling setconfig('pH', 'CO2_Manual_Max_(%)', 100)
15:53:48 Calling setconfig('DO', 'O2_Manual_Max_(%)', 100)
15:53:48 Calling setconfig('DO', 'N2_Manual_Max_(%)', 100)
15:53:49 Calling setag(0, 32)
15:53:49 Calling settemp(0, 37)
Creating new file: 'DO micro test rampdown L plate A 1 170811.csv'
rampdown(0.1, 20, 7200)
15:53:50 Calling setmg(1, 0.1)
15:53:50 Calling setph(1, 100, 0)
15:53:51 Calling setdo(2, 0, 0)

15:53:51 Calling setph(2, 0, 0)
flushheadspace('air', 0.5, 300)
15:53:52 Calling setmg(1, 0.5)
Sleeping 300 seconds, ending at 08/11/2017 03:58:52 PM
15:58:52 Calling setdo(2, 0, 0)
Creating new file: 'DO micro test rampup L plate A 1 170811.csv'
rampup(0.1, 100, 80, 7200)
15:58:52 Calling setmg(1, 0.1)
15:58:52 Calling setdo(1, 0, 100)
15:58:53 Calling setph(2, 100, 0)
15:58:53 Calling setdo(1, 0, 100)
(waitdo_higher) Sleeping 60 seconds from 08/11/2017 05:35:08 PM to 08/11/2017 05:36:08 PM          
17:36:08 Calling setdo(2, 0, 0)
17:36:09 Calling setdo(2, 0, 0)


In [11]:
connect('192.168.1.81')
setup_gases()
setup_controls(32, 37)
setup_logger()
calibrate(0)
chd("Sam Microsparger Test")
for i in range(1,4):
    dotest(0.125, 100, "L plate A 0.125LPM %d"%i)    

12:22:08 Calling setconfig('pH', 'CO2_Manual_Max_(%)', 100)
12:22:09 Calling setconfig('DO', 'O2_Manual_Max_(%)', 100)
12:22:09 Calling setconfig('DO', 'N2_Manual_Max_(%)', 100)
12:22:10 Calling setag(0, 32)
12:22:10 Calling settemp(0, 37)
12:22:11 Calling setmg(1, 0.5)
12:22:12 Calling setdo(1, 0, 100)
12:22:12 Calling setph(2, 0, 0)
12:22:13 Calling setdo(1, 0, 100)
12:22:13 Calling setdo(1, 0, 0)
12:22:14 Calling getRawValue('doa')
12:22:14 Calling trycal('doa', 2424.688477, 100)
12:22:14 Calling savetrialcal('doa')
Creating new file: 'DO micro test rampdown L plate A 0.125LPM 1 170814.csv'
rampdown(0.125, 20, 7200)
12:22:14 Calling setmg(1, 0.125)
12:22:14 Calling setph(1, 100, 0)
12:22:15 Calling setdo(2, 0, 0)
(waitdo_lower) Sleeping 60 seconds from 08/14/2017 01:13:22 PM to 08/14/2017 01:14:22 PM          
13:14:22 Calling setph(2, 0, 0)
flushheadspace('air', 0.5, 300)
13:14:22 Calling setmg(1, 0.5)
Sleeping 300 seconds, ending at 08/14/2017 01:19:22 PM
13:19:22 Calling setdo(2,

In [ ]:
connect('192.168.1.81')
setup_gases()
setup_controls(32, 37)
setup_logger()
calibrate(300)
chd("Sam Microsparger Test")
for i in range(1,4):
    dotest(0.2, 100, "L plate A 0.2LPM %d"%i)    

10:44:31 Calling setconfig('pH', 'CO2_Manual_Max_(%)', 100)
10:44:32 Calling setconfig('DO', 'O2_Manual_Max_(%)', 100)
10:44:32 Calling setconfig('DO', 'N2_Manual_Max_(%)', 100)
10:44:32 Calling setag(0, 32)
10:44:33 Calling settemp(0, 37)
10:44:34 Calling setmg(1, 0.5)
10:44:35 Calling setdo(1, 0, 100)
10:44:36 Calling setph(2, 0, 0)
10:44:36 Calling setdo(1, 0, 100)
Sleeping 150 seconds, ending at 08/15/2017 10:47:06 AM
10:47:06 Calling setdo(1, 0, 0)
Sleeping 150 seconds, ending at 08/15/2017 10:49:37 AM
10:49:37 Calling getRawValue('doa')
10:49:37 Calling trycal('doa', 99.245193, 100)
10:49:37 Calling savetrialcal('doa')
Creating new file: 'DO micro test rampdown L plate A 0.2LPM 1 170815.csv'
rampdown(0.2, 20, 7200)
10:49:37 Calling setmg(1, 0.2)
10:49:37 Calling setph(1, 100, 0)
10:49:38 Calling setdo(2, 0, 0)
(waitdo_lower) Sleeping 60 seconds from 08/15/2017 12:48:58 PM to 08/15/2017 12:49:58 PM          
12:49:58 Calling setph(2, 0, 0)
flushheadspace('air', 0.5, 300)
12:49:58 

In [7]:
connect('192.168.1.81')
setup_gases()
setup_controls(32, 37)
setup_logger()
mkfldr("Sam Microsparger Test\\0.1-0.2 overnight 170815")
calibrate(3600*3)
for i in ntests(3):
    for mg in (0.1, 0.125, 0.15, 0.175, 0.2):
        dotest(mg, 100, "L plate A %.3fLPM %d"%(mg,i))    

16:45:17 Calling setconfig('pH', 'CO2_Manual_Max_(%)', 100)
16:45:19 Calling setconfig('DO', 'O2_Manual_Max_(%)', 100)
16:45:19 Calling setconfig('DO', 'N2_Manual_Max_(%)', 100)
16:45:20 Calling setag(0, 32)
16:45:20 Calling settemp(0, 37)
16:45:21 Calling setmg(1, 0.5)
16:45:22 Calling setdo(1, 0, 100)
16:45:22 Calling setph(2, 0, 0)
16:45:23 Calling setdo(1, 0, 100)
Sleeping 7200 seconds, ending at 08/15/2017 06:45:23 PM
18:45:24 Calling setdo(1, 0, 0)
Sleeping 3600 seconds, ending at 08/15/2017 07:45:24 PM
19:45:24 Calling getRawValue('doa')
19:45:25 Calling trycal('doa', 100.156754, 100)
19:45:25 Calling savetrialcal('doa')
Creating new file: 'DO micro test rampdown L plate A 0.100LPM 1 170815.csv'
rampdown(0.1, 20, 7200)
19:45:25 Calling setmg(1, 0.1)
19:45:25 Calling setph(1, 100, 0)
19:45:26 Calling setdo(2, 0, 0)
(waitdo_lower) Sleeping 60 seconds from 08/15/2017 08:34:32 PM to 08/15/2017 08:35:32 PM          
20:35:32 Calling setph(2, 0, 0)
flushheadspace('air', 0.5, 300)
20:3

(waitdo_lower) Sleeping 60 seconds from 08/16/2017 04:28:52 PM to 08/16/2017 04:29:52 PM          
16:29:53 Calling setph(2, 0, 0)
flushheadspace('air', 0.5, 300)
16:29:53 Calling setmg(1, 0.5)
16:29:54 Calling setdo(2, 0, 0)
Sleeping 300 seconds, ending at 08/16/2017 04:34:54 PM
16:34:54 Calling setdo(2, 0, 0)
Creating new file: 'DO micro test rampup L plate A 0.175LPM 2 170816.csv'
rampup(0.175, 100, 80, 7200)
16:34:55 Calling setmg(1, 0.175)
16:34:55 Calling setdo(1, 0, 100)
16:34:55 Calling setph(2, 100, 0)
16:34:56 Calling setdo(1, 0, 100)
(waitdo_higher) Sleeping 60 seconds from 08/16/2017 05:44:08 PM to 08/16/2017 05:45:08 PM          
17:45:08 Calling setdo(2, 0, 0)
17:45:08 Calling setdo(2, 0, 0)
Creating new file: 'DO micro test rampdown L plate A 0.200LPM 2 170816.csv'
rampdown(0.2, 20, 7200)
17:45:09 Calling setmg(1, 0.2)
17:45:09 Calling setph(1, 100, 0)
17:45:09 Calling setdo(2, 0, 0)
(waitdo_lower) Sleeping 60 seconds from 08/16/2017 07:42:29 PM to 08/16/2017 07:43:29 PM

In [ ]:
connect('192.168.1.81')
setup_gases()
setup_controls(32, 37)
setup_logger()
mkfldr("Sam Microsparger Test\\L Plate C 0.1-0.2 overnight 170817")
calibrate(3600*3)
for i in ntests(3):
    for mg in (0.1, 0.125, 0.15, 0.175, 0.2):
        dotest(mg, 100, "L plate A %.3fLPM %d"%(mg,i))    

11:21:42 Calling setconfig('pH', 'CO2_Manual_Max_(%)', 100)
11:21:43 Calling setconfig('DO', 'O2_Manual_Max_(%)', 100)
11:21:43 Calling setconfig('DO', 'N2_Manual_Max_(%)', 100)
11:21:44 Calling setag(0, 32)
11:21:44 Calling settemp(0, 37)
11:21:45 Calling setmg(1, 0.5)
11:21:47 Calling setdo(1, 0, 100)
11:21:47 Calling setph(2, 0, 0)
11:21:47 Calling setdo(1, 0, 100)
Sleeping 7200 seconds, ending at 08/17/2017 01:21:48 PM
13:21:48 Calling setdo(1, 0, 0)
Sleeping 3600 seconds, ending at 08/17/2017 02:21:48 PM
14:21:49 Calling getRawValue('doa')
14:21:49 Calling trycal('doa', 98.389549, 100)
14:21:50 Calling savetrialcal('doa')
Creating new file: 'DO micro test rampdown L plate A 0.100LPM 1 170817.csv'
rampdown(0.1, 20, 7200)
14:21:50 Calling setmg(1, 0.1)
14:21:50 Calling setph(1, 100, 0)
14:21:51 Calling setdo(2, 0, 0)
(waitdo_lower) Sleeping 60 seconds from 08/17/2017 03:11:00 PM to 08/17/2017 03:12:00 PM          
15:12:00 Calling setph(2, 0, 0)
flushheadspace('air', 0.5, 300)
15:12

In [ ]:
z